# Injected State

While working with [RAGV4](https://github.com/MatteoFalcioni/experiments/blob/main/RAG_V4.0_.ipynb) I noticed a huge problem. 

I was building an agent supervisor system that managed two worker agents - a data analyst and a visualizer. Problem was, when the data analyst had completed its analysis on the datasets, the visualizer wasn't able to access the analysed data, and he would try to run the analysis again. 

Why did this happen? Well, the datasets were loaded in memory and stored as global variables in a dictionary. But this made it difficult for different agents to retrieve the same data, as the global needed to be shared and the possibility to access it needed to be enforced through prompting. Isn't there a standard way of letting all agents access the same data at runtime?

Of course there is, and it's actually a basic concept in LangGraph: it's the graph's **State**. If we want some object to be accessible and visible to all agents at any time, than that object should be in the Graph state.

So we'd just need to subclass the basic `MessagesState` class from LangGraph and add our relevant object to state to define our *"state schema"*, like this: 

```python
class MyState(MessagesState):   # already contains a structure like Annotated[Sequence[BaseMessage], operator.add]
    dataframes : dict   # add your needed fields 
```

Perfect, right...? No! If we only did this and tried to access the `dataframes` dict inside our tools, we wouldn't manage to do so. 

We have to follow a specific syntax in order to access state data in our tools and modify it. We need to use the [`InjectedState`](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.tool_node.InjectedState) annotation.

### Using `InjectedState` in tools

The following is an example from the [`InjectedState`](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.tool_node.InjectedState) documentation.

Here they don't subclass `MessagesState` but the principle is the same. 

In [1]:
from typing import List
from typing_extensions import Annotated, TypedDict

from langchain_core.messages import BaseMessage, AIMessage
from langchain_core.tools import tool

from langgraph.prebuilt import InjectedState, ToolNode


class AgentState(TypedDict):    # create your state schema
    messages: List[BaseMessage]
    foo: str

@tool
def state_tool(x: int, state: Annotated[dict, InjectedState]) -> str:   # use Annotated[dict, InjectedState]
    '''Do something with state.'''
    if len(state["messages"]) > 2:      # here we use the whole state
        return state["foo"] + str(x)
    else:
        return "not enough messages"

@tool
def foo_tool(x: int, foo: Annotated[str, InjectedState("foo")]) -> str: # we can select a specific field to pass with InjectedState("<field_name>")
    '''Do something else with state.'''
    return foo + str(x + 1)

node = ToolNode([state_tool, foo_tool])

tool_call1 = {"name": "state_tool", "args": {"x": 1}, "id": "1", "type": "tool_call"}
tool_call2 = {"name": "foo_tool", "args": {"x": 1}, "id": "2", "type": "tool_call"}
state = {
    "messages": [AIMessage("", tool_calls=[tool_call1, tool_call2])],
    "foo": "bar",
}
node.invoke(state)

{'messages': [ToolMessage(content='not enough messages', name='state_tool', tool_call_id='1'),
  ToolMessage(content='bar2', name='foo_tool', tool_call_id='2')]}

### Integrating `InjectedState` with agents

The simplest way to integrate `InjectedState` with agentic framework is to use the [`create_react_agent()`](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent) function from LangGraph. 

We need to pass our custom state as the `state_schema` parameter, like this:

```python
agent = create_react_agent(
    model=..., 
    tools=[state_tool, foo_tool],
    state_schema=MyState
)
```

In this way the model knows what states it's working with.

### Other Context Management practises 

Before moving to a practical example, allow us to cite [other common context management practises in LangGraph](https://langchain-ai.github.io/langgraph/how-tos/tool-calling/#context-management).

As a matter of fact, `InjectedState` is not the only way to allow our graph state to persist as context data. It is the most flexible and "lightweight" standard, but we can use: 

* [`Configuration`](https://langchain-ai.github.io/langgraph/how-tos/tool-calling/#configuration) : "*Use configuration when you have static, immutable runtime data that tools require, such as user identifiers"*. 

* [Long term memory](https://langchain-ai.github.io/langgraph/how-tos/tool-calling/#long-term-memory) : *"Use long-term memory to store user-specific or application-specific data across different sessions"*. The main difference here is that the data persists to other sources (like disk) even after the current session ends. This is useful for working with heavy datasets, or to leverage memory across different runs of a chatbot. See [Stores](https://langchain-ai.github.io/langgraph/reference/store/#storage) for further references.

## Example #1 of `InjectedState` workflow

Let's make a simple example to recap the actual workflow with `InjectedState`:

### 1. Create your custom "`state_schema`":

In [2]:
from langgraph.graph import MessagesState

class CustomState(MessagesState): 
    username : str 
    remaining_steps : int

>**Note:** from the [create_react_agent() doc](https://langchain-ai.github.io/langgraph/reference/agents/#langgraph.prebuilt.chat_agent_executor.create_react_agent):  
>
>*`state_schema` : An optional state schema that defines graph state. Must have `messages` and `remaining_steps` keys. Defaults to AgentState that defines those two keys.*
>
> `messages` is implemented by `MessagesState`, but we need to implement `reamining_steps` otherwise it will error.

### 2. Write your tools using `InjectedState`:

In [6]:
from langchain_core.tools import tool
from langgraph.prebuilt import InjectedState
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool, InjectedToolCallId
from langgraph.types import Command

@tool 
def get_internal_value(state : Annotated[CustomState, InjectedState]) -> str:
    """tool to retrieve the username"""
    return state.get('username')

@tool 
def update_username(new_name : str, tool_call_id : Annotated[str, InjectedToolCallId]
) -> Command:
    """Update username in short-term memory."""
    
    return Command(update={
        "username" : new_name,
        "messages" : [
            ToolMessage(f"Updated username to {new_name}", tool_call_id=tool_call_id)
        ]
    })

>**Note:** Notice how we used: 
>   - `state.get()` to read the value
>   - a `Command` return in order to update the state : here we also need to append to messages a `ToolMessage`, otherwise it will error. In order to do so, we constructed it with `Annotated[str, InjectedToolCallId]` to follow the correct approach - but we could have done it in a simpler way like `ToolMessage("Success", tool_call_id=...)` as the error suggests:
>
>   *Expected to have a matching ToolMessage in Command.update for tool 'update_username', got: []. Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage. You can fix it by modifying the tool to return `Command(update=[ToolMessage("Success", tool_call_id=tool_call_id), ...], ...)`*

### 3. Create the agent passing the custom `state_schema` 

In [7]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

agent = create_react_agent(
    model=ChatOpenAI(model="gpt-4o"),
    tools=[update_username, get_internal_value],
    state_schema=CustomState
)

In [8]:
from langchain_core.messages import HumanMessage

initial_state = {
    "messages": [HumanMessage(content="Whats my username? Update it to Mario. What's the username now?")],
    "username": "Matteo",
    "remaining_steps": 15
}

print(agent.invoke(initial_state)["messages"][-1].content)

Your previous username was "Matteo." It has been updated to "Mario." Now, your username is "Mario."


## RAGV4 Example

How about a practical application? 

Let's correctly build RAGV4 using `InjectedState`. We will only build the data analyst.